<a href="https://colab.research.google.com/github/Chadliu123/TibameAI/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"-".join(map(str, [1, 2, 3, 4, 5]))

'1-2-3-4-5'

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 3s 0us/step


In [ ]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd

def read(path):
    with open(path, "r", encoding="utf-8") as f:
      content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

traindf = get_data("train")
testdf = get_data("test")
testdf

In [ ]:
TOK = 3000
LEN = 512
EM = 128

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform的fit
tok.fit_on_texts(traindf["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(traindf["contents"])
x_test_seq = tok.texts_to_sequences(testdf["contents"])

In [ ]:
# tok.word_index
tok.index_word[2]

'and'

In [ ]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(TOK+1, EM, mask_zero=True, input_length=LEN),
  GlobalAveragePooling1D(),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
y_train = traindf["sentiment"]
y_test = testdf["sentiment"]

In [ ]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 5s - loss: 0.4822 - accuracy: 0.7609 - val_loss: 0.4393 - val_accuracy: 0.8092
Epoch 2/50
225/225 - 5s - loss: 0.1709 - accuracy: 0.9366 - val_loss: 0.3355 - val_accuracy: 0.8604
Epoch 3/50
225/225 - 5s - loss: 0.0387 - accuracy: 0.9913 - val_loss: 0.4962 - val_accuracy: 0.8472
Epoch 4/50
225/225 - 5s - loss: 0.0062 - accuracy: 0.9995 - val_loss: 0.6840 - val_accuracy: 0.8300
Epoch 5/50
225/225 - 5s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6879 - val_accuracy: 0.8424
Epoch 6/50
225/225 - 5s - loss: 6.0663e-04 - accuracy: 1.0000 - val_loss: 0.7987 - val_accuracy: 0.8320
Epoch 7/50
225/225 - 5s - loss: 3.5011e-04 - accuracy: 1.0000 - val_loss: 0.7900 - val_accuracy: 0.8388


In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3146 - accuracy: 0.8687


[0.3146132528781891, 0.8687199950218201]

In [ ]:
layers = [
  Embedding(TOK+1, EM, mask_zero=True),
  GlobalAveragePooling1D()
]
w = model.layers[0].get_weights()
infer = Sequential(layers)
infer.layers[0].set_weights(w)
infer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# w
# infer.layers[0].get_weights()
# 25000, 512
# [512list, 512list]
target = "the"
# [1list]
pre = infer.predict([[tok.word_index[target]]])
pre[0]

array([-0.01413893, -0.02599616,  0.04677478, -0.0239989 ,  0.04429387,
        0.00356539,  0.04542689, -0.03761591, -0.02235222,  0.00871463,
       -0.00211833, -0.00362252, -0.00377093,  0.03827894,  0.0457781 ,
        0.03480729, -0.01083549,  0.03760238,  0.04094679, -0.01250048,
        0.03123139,  0.01425325,  0.04426681,  0.0460353 , -0.02688274,
        0.03817122,  0.04568637,  0.01241342,  0.03202368, -0.00508547,
       -0.01276767, -0.02725678, -0.04598273, -0.03685294,  0.04527522,
       -0.03414955,  0.01529148,  0.01422014,  0.03154403, -0.0176141 ,
        0.04339239,  0.03712037, -0.04213715, -0.02416993,  0.03686471,
        0.00011513,  0.0380785 ,  0.01568823, -0.02032567, -0.0111946 ,
        0.04858376, -0.01775031,  0.02428801, -0.04429708,  0.0137257 ,
       -0.02177798,  0.00864636, -0.04281788, -0.04455842, -0.0277067 ,
        0.03669086, -0.02449844, -0.02120966,  0.01028173,  0.04728096,
        0.00080613,  0.00246583, -0.04640976, -0.04182769,  0.03

In [ ]:
f = open("vec.txt", "w", encoding="utf-8")
f.write("{} {}\n".format(TOK, EM))
for i in range(1, TOK+1):
    # [1list]
    pre = infer.predict([[i]])[0]
    v = " ".join(map(str, pre))
    f.write("{} {}\n".format(tok.index_word[i], v))
f.close()

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("vec.txt")

In [ ]:
w2v.wv.vocab

In [ ]:
# 詞彙列表: w2v.wv.vocab
w2v.most_similar("girl")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('again', 0.30507028102874756),
 ('trying', 0.28944408893585205),
 ('inept', 0.28015726804733276),
 ('able', 0.26081621646881104),
 ('technically', 0.25740694999694824),
 ('regret', 0.25650230050086975),
 ('dreadful', 0.251526802778244),
 ('strangely', 0.24714136123657227),
 ('they', 0.2462022602558136),
 ('correct', 0.24391251802444458)]